# 2. Продвинутые операции

In [2]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['HADOOP_USER_NAME'] = 'root'  # Обход проверки пользователя

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Test") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

## 2.1 Оконные функции

Оконные функции (Window Functions) в PySpark позволяют выполнять вычисления над группами строк, сохраняя при этом индивидуальность каждой строки.

Основные компоненты оконных функций:
1. Оконная спецификация (WindowSpec) - определяет, какие строки будут включены в рамки окна для каждой строки
2. Оконная функция - функция, которая применяется к данным в рамках окна

**Ранжирование** (rank(), dense_rank(), row_number(), percent_rank())  
**Агрегатные функции** (sum(), avg(), max(), min())  
**Смещение** (lag(), lead())  
**Аналитические функции** (first(), last(), cume_dist(), ntile())  

In [ ]:
# Задача: Найти разницу между текущей и предыдущей покупкой для каждого пользователя.

# Создаём DataFrame с покупками
sales = spark.createDataFrame([
    (1, "2023-01-10", 100),
    (1, "2023-01-15", 200),
    (2, "2023-01-12", 50),
    (1, "2023-01-20", 300)
], ["user_id", "date", "amount"])

# Определяем окно
from pyspark.sql.window import Window
window = Window.partitionBy("user_id").orderBy("date")

# Добавляем разницу с предыдущей покупкой
from pyspark.sql.functions import lag, col
sales_with_diff = sales.withColumn("prev_amount", lag("amount").over(window)).withColumn("diff", col("amount") - col("prev_amount"))
sales_with_diff.show()

In [ ]:
# Топ-3 товара по категориям

# Данные о товарах
products = spark.createDataFrame([
    (1, "Laptop", "Electronics", 999),
    (2, "Phone", "Electronics", 699),
    (3, "Desk", "Furniture", 200),
    (4, "Chair", "Furniture", 150)
], ["product_id", "name", "category", "price"])

# Окно для ранжирования
window = Window.partitionBy("category").orderBy(col("price").desc())

# Топ-3 в каждой категории
from pyspark.sql.functions import dense_rank
top_products = products.withColumn("rank", dense_rank().over(window)).filter(col("rank") <= 3)
top_products.show()

## 2.2 Работа с датами и строками

Обработка дат и времени:
- Парсинг строк в даты: to_date(), to_timestamp()
- Извлечение компонентов: year(), month(), dayofweek()
- Арифметика: datediff(), date_add(), months_between()
- Форматирование: date_format()

In [ ]:
# Анализ продаж по времени

spark = SparkSession.builder.appName("Data").getOrCreate()

# Создаём DataFrame с датами
sales_data = [
    (1, "2023-01-15", 150.0),
    (2, "15-02-2023", 200.0),  # Нестандартный формат
    (3, "2023/03/20", 99.99)
]
df = spark.createDataFrame(sales_data, ["order_id", "order_date", "amount"])

# Приводим даты к единому формату (yyyy-MM-dd)
df = df.withColumn("parsed_date", 
      to_date(col("order_date"), "yyyy-MM-dd")) \
      .withColumn("parsed_date", 
      coalesce(col("parsed_date"), 
               to_date(col("order_date"), "dd-MM-yyyy"),
               to_date(col("order_date"), "yyyy/MM/dd")))

# Извлекаем год, месяц и день недели
df = df.withColumn("year", year("parsed_date")) \
       .withColumn("month", month("parsed_date")) \
       .withColumn("day_of_week", dayofweek("parsed_date"))

df.show()

In [ ]:
# Расчёт дней между заказами
from pyspark.sql.window import Window

window = Window.partitionBy("year").orderBy("parsed_date")
df = df.withColumn("days_since_last_order", 
      datediff(col("parsed_date"), lag("parsed_date").over(window)))

df.show()

Работа со строками:
- Базовые операции: concat(), substring(), trim()
- Регулярные выражения: regexp_extract(), regexp_replace()
- Проверки: startswith(), endswith(), contains()

In [ ]:
# Очистка и анализ текста

# Извлечение домена из email
users = spark.createDataFrame([
    (1, "alice@example.com"),
    (2, "bob@gmail.com")
], ["user_id", "email"])

users = users.withColumn("domain", 
      regexp_extract(col("email"), "@(.+)$", 1))

users.show()

In [ ]:
# Замена цензурой нецензурных слов
comments = spark.createDataFrame([
    (1, "This is bad!"),
    (2, "Worst product ever")
], ["comment_id", "text"])

bad_words = ["bad", "worst"]
pattern = "|".join(bad_words)
comments = comments.withColumn("clean_text", 
      regexp_replace(col("text"), pattern, "***"))

comments.show()

In [ ]:
# Загрузка CSV с заголовком и выводом схемы
df_Titanic = spark.read.csv("data/train.csv", header=True, inferSchema=True)

# Разделяем "Surname, Title Name" и извлекаем фамилию
df_Titanic = df_Titanic.withColumn("surname", 
      split(col("Name"), ",")[0])

# Длина имени
df_Titanic = df_Titanic.withColumn("name_length", 
      length(col("Name")))

df_Titanic.show(5)

## 2.3 Оптимизация (партиционирование, кэширование)

Партиционирование — разделение данных на части (партиции) для параллельной обработки. Большие файлы (например, 100+ ГБ CSV/Parquet). Частые фильтры по определённым колонкам (например, по дате).

Пример
```
# Запись данных с партиционированием по колонке 'Pclass' (Titanic dataset)
df.write.partitionBy("Pclass").parquet("data/titanic_partitioned")

# Чтение только партиции Pclass=1 (ускоряет загрузку в 3+ раза)
df_pclass1 = spark.read.parquet("data/titanic_partitioned/Pclass=1")
```

```
# Репартиционирование в 200 партиций (для равномерного распределения)
df_repartitioned = df.repartition(200)

# Партиционирование по колонке + указание числа партиций
df_repartitioned = df.repartition(10, "Pclass")
```

Кэширование сохраняет данные в памяти или на диске для повторного использования. Если DataFrame используется более 2-х раз. Перед многократными итерациями (например, в ML-циклах).  
Уровни хранения:
- `MEMORY_ONLY` — только в памяти (быстро, но риск OOM).
- `MEMORY_AND_DISK` — сначала память, потом диск (надежнее).
- `DISK_ONLY` — только диск (медленно, но безопасно).

Пример
```
from pyspark.storagelevel import StorageLevel

# Кэширование в памяти (аналог .cache())
df.cache()  

# Явное указание уровня
df.persist(StorageLevel.MEMORY_AND_DISK)

# Проверка статуса
df.is_cached  # -> True

# Освобождение памяти
df.unpersist()
```

```
# Плохо: DataFrame читается 2 раза
df.filter(df.Age > 30).count()
df.filter(df.Age > 30).show()

# Хорошо: Кэшируем перед повторным использованием
filtered_df = df.filter(df.Age > 30).persist()
filtered_df.count()
filtered_df.show()
```

### Настройка памяти и параметров Spark

```
spark = SparkSession.builder \
    .appName("Optimization") \
    .config("spark.executor.memory", "4g") \  # Память на executor
    .config("spark.driver.memory", "2g") \    # Память на driver
    .config("spark.sql.shuffle.partitions", "200") \  # Число партиций после shuffle
    .master("local[*]") \
    .getOrCreate()
```

Shuffle — перераспределение данных между узлами (например, при groupBy или join).  
Как уменьшить нагрузку:
- Увеличьте spark.sql.shuffle.partitions (по умолчанию 200).
- Используйте broadcast для маленьких таблиц:
```
from pyspark.sql.functions import broadcast
df1.join(broadcast(df2), "key")
```

Мониторинг и отладка. Откройте в браузере: `http://localhost:4040` (для локального режима).

| Ошибка | Причина | Решение |
| -- | -- | -- |
| OutOfMemoryError | Нехватка памяти | Увеличьте spark.executor.memory |
| Data skew | Неравномерное распределение данных | Используйте repartition или salting |
| Slow JOIN | Большие таблицы без оптимизации | Примените broadcast для малых таблиц |

## 2.4 UDF (пользовательские функции)

User-Defined Functions — это пользовательская функция, которую можно применять к колонкам DataFrame.

Когда встроенных функций Spark недостаточно. Для применения сложной Python-логики (например, NLP, машинное обучение).

UDF работают медленнее встроенных функций (из-за накладных расходов на сериализацию). Для ускорения используют Pandas UDF (векторизованные функции).

In [ ]:
# Функция для категоризации возраста.
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName("UDFs").getOrCreate()

# Данные
data = [("Alice", 25), ("Bob", 60), ("Charlie", 15)]
df = spark.createDataFrame(data, ["name", "age"])

# Шаг 1: Создаём Python-функцию
def age_category(age):
    if age < 18:
        return "child"
    elif age >= 60:
        return "senior"
    else:
        return "adult"

# Шаг 2: Регистрируем UDF (указываем тип возвращаемого значения)
age_category_udf = udf(age_category, StringType())

# Шаг 3: Применяем к DataFrame
df = df.withColumn("age_category", age_category_udf(df["age"]))
df.show()

Векторизованные Pandas UDF (быстрые). Pandas UDF работают в 10-100 раз быстрее обычных UDF, т.к. обрабатывают данные партиями.

In [ ]:
# Расчёт квадрата числа через Pandas UDF.

from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import IntegerType
import pandas as pd

# Шаг 1: Создаём векторную функцию
@pandas_udf(IntegerType())
def square_udf(age: pd.Series) -> pd.Series:
    return age ** 2

# Шаг 2: Применяем
df = df.withColumn("age_squared", square_udf(df["age"]))
df.show()

Кэширование результатов. Если UDF вызывается многократно, закэшируйте DataFrame: `df.cache()`

In [ ]:
# UDF, которая преобразует колонку Name в формат ФАМИЛИЯ, 
# Имя (например, "Braund, Mr. Owen Harris" → "BRAUND, Owen").
def format_name(name):
    surname = name.split(",")[0].upper()
    first_name = name.split(",")[1].split(".")[1].strip()
    return f"{surname}, {first_name}"

format_name_udf = udf(format_name, StringType())
df_Titanic = df_Titanic.withColumn("formatted_name", format_name_udf(df_Titanic["Name"]))

# Pandas UDF для расчёта длины строки в колонке Name.
@pandas_udf(IntegerType())
def name_length_udf(names: pd.Series) -> pd.Series:
    return names.str.len()

df_Titanic = df_Titanic.withColumn("name_length", name_length_udf(df_Titanic["Name"]))

# 3. Тест скорости
import time

start = time.time()
df.withColumn("name_length", name_length_udf(df["Name"])).count()
print(f"Pandas UDF: {time.time() - start} sec")

start = time.time()
df.withColumn("formatted_name", format_name_udf(df["Name"])).count()
print(f"Regular UDF: {time.time() - start} sec")

## 2.5 Чтение/запись в разных форматах

| Формат | Описание | Плюсы | Минусы |
| - | - | - | - |
| CSV | Текстовый формат с разделителями | Простота, человекочитаемость | Нет схемы, медленный |
| JSON | Полуструктурированный формат | Поддержка вложенных данных | Медленный, большой объём |
| Parquet | Бинарный колоночный формат | Быстрый, сжатый, с поддержкой схемы | Нечитаем для человека |
| Avro | Бинарный формат с схемой | Эффективен для сериализации | Требует схемы |

In [ ]:
#spark.stop()